# DiracPy Tutorial - Part 1

## Quantum systems in diracpy

The `qsys` class in diracpy creates quantum systems around a given Hamiltonian. To understand what a diracpy `qsys` is and why it is needed, we need to first explain the front-end and back-end of diracpy. The front-end presents the user with the interface to define bras, kets and operators. This was covered in part 1 of this tutorial, and allows these objects to be coded in the same way they would be written in Dirac notation. The back-end supports several methods to solve the quantum dynamics under a given Hamiltonian.

Ultimately, to solve the dynamics, the abstract dirac vectors and operators must be written explicitly using lineary algebra (using numpy arrays in our case). The `qsys` class takes a Hamiltonian and the inital state of the system, then finds an appropriate basis to describe the system, and defines the Hamiltonian matrix in this basis.